In [1]:
%%capture 
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install sentencepiece
!pip install tweet-preprocessor

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import re
import preprocessor as p
def clean_data(sentence):
   # sentence = sentence.encode().decode("unicode_escape")
    sentence = sentence.lower()
    sentence = re.sub(r'\n'," ",sentence)
    sentence = p.tokenize(sentence)
    return sentence

In [7]:
df.label.value_counts()


0    530
1    398
Name: label, dtype: int64

In [5]:
import pandas as pd

df = pd.read_csv("/kaggle/input/d/syairdafiq/indonesia-hoax-dataset/dataset_combine_1_2.csv")
df['text'] = df['text'].apply(lambda x: clean_data(x))
df['len'] = df['text'].apply(lambda x:len(x.split(" ")))
df = df[df.len <= 384][['text','label']]

from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df,stratify = df.label,random_state = 42,test_size = 0.2)
df_train.shape,df_test.shape

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


((742, 2), (186, 2))

In [6]:
df.label.value_counts()

0    530
1    398
Name: label, dtype: int64

In [7]:
from datasets import Dataset,DatasetDict
from transformers import BertTokenizer, DataCollatorWithPadding,AutoModel,AutoTokenizer

raw_datasets = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "test": Dataset.from_pandas(df_test)
    })
raw_datasets = raw_datasets.remove_columns(["__index_level_0__"])
checkpoint = "cahya/distilbert-base-indonesian"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"],padding = True, truncation=True,max_length = 384)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label","labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'attention_mask']

In [9]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=16, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=1, collate_fn=data_collator
)

In [10]:
import gc
gc.collect()

87

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at cahya/distilbert-base-indonesian were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at cahya/distilbert-base-indonesian and are newly initialized: ['classifier.bias', 'pre_classifier.bi

In [17]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([16]),
 'input_ids': torch.Size([16, 384]),
 'attention_mask': torch.Size([16, 384])}

In [18]:
from transformers import Trainer
from datasets import load_metric
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback, IntervalStrategy
from datasets import load_metric
import numpy as np


training_args = TrainingArguments(
    evaluation_strategy = IntervalStrategy.STEPS,
    save_total_limit = 2,
    fp16 = True,
    output_dir="/kaggle/working/DistilBERT-Hoax-Detection",
    logging_steps=2,
    save_steps=93,
    eval_steps=93,
    num_train_epochs=10,
    metric_for_best_model = 'accuracy',
    load_best_model_at_end=True,
    warmup_ratio = 0.15,
    learning_rate = 5e-6,
    push_to_hub = True,
    overwrite_output_dir = True
)
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions[0], axis=1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

/kaggle/working/DistilBERT-Hoax-Detection is already a clone of https://huggingface.co/dafqi/DistilBERT-Hoax-Detection. Make sure you pull the latest changes with `repo.git_pull()`.


In [19]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
93,0.664400,0.636804,0.623656
186,0.415100,0.529957,0.725806
279,0.364500,0.500344,0.768817
372,0.328300,0.458473,0.795699
465,0.255700,0.459859,0.806452
558,0.399300,0.500403,0.806452
651,0.053600,0.465829,0.838710
744,0.194400,0.526409,0.827957
837,0.061200,0.519514,0.838710
930,0.060200,0.526112,0.844086


TrainOutput(global_step=930, training_loss=0.32538604093815693, metrics={'train_runtime': 411.3462, 'train_samples_per_second': 18.038, 'train_steps_per_second': 2.261, 'total_flos': 737181073520640.0, 'train_loss': 0.32538604093815693, 'epoch': 10.0})

In [20]:
trainer.push_to_hub("End of Training")

To https://huggingface.co/dafqi/DistilBERT-Hoax-Detection
   40f0a2a..00d94de  main -> main

To https://huggingface.co/dafqi/DistilBERT-Hoax-Detection
   00d94de..4d83eba  main -> main



'https://huggingface.co/dafqi/DistilBERT-Hoax-Detection/commit/00d94de8ce19ff62d24ed945e64052f0a1b73bba'

In [31]:
model.config.id2label = {0 : "Valid", 1: "Hoaks"}
model.config.label2id = {"Valid" : 0,'Hoaks':1}

repo_name = "dafqi/DistilBERT-Hoax-Detection"
model.config.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/dafqi/DistilBERT-Hoax-Detection/commit/fae3f06643fba4049cbf6aea2f8c8321dfe345d0', commit_message='Upload config', commit_description='', oid='fae3f06643fba4049cbf6aea2f8c8321dfe345d0', pr_url=None, pr_revision=None, pr_num=None)

In [40]:
from transformers import pipeline

classifier = pipeline("text-classification", 'dafqi/DistilBERT-Hoax-Detection')
classifier(df.iloc[0]['text'])

[{'label': 'Valid', 'score': 0.9937870502471924}]

'jakarta, di jejaring sosial, banyak beredar informasi yang menyebut lele sebagai ikan paling jorok. dalam sesuap daging ikan lele, terkandung $NUMBER$ sel kanker. benarkah? julukan sebagai ikan paling jorok merujuk pada sifat lele yang doyan mengonsumsi segala jenis limbah di perairan. bahkan sebuah artikel yang cukup viral di internet menyebutkan kotoran manusia juga dijadikan pakan pada sebuah budidaya lele di kota haikou, china. sementara itu di habitat aslinya, lele atau catfish juga dikenal sebagai spesies ikan yang sangat tangguh. ikan ini dilengkapi alat pernapasan tambahan berupa labirin, sehingga mampu bertahan hidup dalam kondisi perairan berlumpur atau bahkan tercemar. agaknya, fakta inilah yang memunculkan dugaan soal akumulasi racun karsinogen (penyebab kanker) di tubuh ikan lele. untungnya, ikan lele yang beredar di pasaran bukan berasal dari alam liar. lele banyak dibudidayakan di kolam-kolam, yang mestinya bisa dikendalikan agar bebas dari pencemaran. pakan yang diberi

In [18]:
trainer.push_to_hub(
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.push_to_hub()                                                                        │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:3716 in push_to_hub              │
│                                                                                                  │
│   3713 │   │   # If a user calls manually `push_to_hub` with `self.args.push_to_hub = False`, w  │
│   3714 │   │   # it might fail.                                                                  │
│   3715 │   │   if not hasattr(self, "repo"):                                                     │
│ ❱ 3716 │   │   │   self.init_git_repo()                                                          │
│   3717 │   │                                                                                     │
│   3718 │   │   model_name = kwargs.pop("model_name", None)                                       │
│   3719 │   │   if model_name is None and self.args.should_save:                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:3571 in init_git_repo            │
│                                                                                                  │
│   3568 │   │   # Make sure the repo exists.                                                      │
│   3569 │   │   create_repo(repo_name, token=self.args.hub_token, private=self.args.hub_private_  │
│   3570 │   │   try:                                                                              │
│ ❱ 3571 │   │   │   self.repo = Repository(self.args.output_dir, clone_from=repo_name, token=sel  │
│   3572 │   │   except EnvironmentError:                                                          │
│   3573 │   │   │   if self.args.overwrite_output_dir and at_init:                                │
│   3574 │   │   │   │   # Try again after wiping output_dir                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_validators.py:118 in _inner_fn    │
│                                                                                                  │
│   115 │   │   if check_use_auth_token:                                                           │
│   116 │   │   │   kwargs = smoothly_deprecate_use_auth_token(fn_name=fn.__name__, has_token=ha   │
│   117 │   │                                                                                      │
│ ❱ 118 │   │   return fn(*args, **kwargs)                                                         │
│   119 │                                                                                          │
│   120 │   return _inner_fn  # type: ignore                                                       │
│   121                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/huggingface_hub/repository.py:516 in __init__            │
│                                                                                                  │
│    513 │   │   │   self.huggingface_token = HfFolder.get_token()                                 │
│    514 │   │                                               

In [68]:
predictions = trainer.predict(tokenized_datasets["test"])

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 predictions = trainer.predict(tokenized_datasets["test"])                                    │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:3129 in predict                  │
│                                                                                                  │
│   3126 │   │   start_time = time.time()                                                          │
│   3127 │   │                                                                                     │
│   3128 │   │   eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else se  │
│ ❱ 3129 │   │   output = eval_loop(                                                               │
│   3130 │   │   │   test_dataloader, description="Prediction", ignore_keys=ignore_keys, metric_k  │
│   3131 │   │   )                                                                                 │
│   3132 │   │   total_batch_size = self.args.eval_batch_size * self.args.world_size               │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:3353 in evaluation_loop          │
│                                                                                                  │
│   3350 │   │   │   │   │   EvalPrediction(predictions=all_preds, label_ids=all_labels, inputs=a  │
│   3351 │   │   │   │   )                                                                         │
│   3352 │   │   │   else:                                                                         │
│ ❱ 3353 │   │   │   │   metrics = self.compute_metrics(EvalPrediction(predictions=all_preds, lab  │
│   3354 │   │   else:                                                                             │
│   3355 │   │   │   metrics = {}                                                                  │
│   3356                                                                                           │
│                                                                                                  │
│ in compute_metrics:29                                                                            │
│                                                                                                  │
│   26 def compute_metrics(eval_pred):                                                             │
│   27 │   predictions, labels = eval_pred                                                         │
│   28 │   predictions = np.argmax(predictions[0], axis=1)                                         │
│ ❱ 29 │   return metric.compute(predictions=predictions, references=labels)                       │
│   30                                                                                             │
│   31 trainer = Trainer(                                                                          │
│   32 │   model,                                                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'metric' is not defined

In [69]:
predictions

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 predictions                                                                                  │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predictions' is not defined

In [70]:
from sklearn.metrics import classification_report

print(classification_report(predictions.label_ids,np.argmax(predictions.predictions[0],axis = 1)))

              precision    recall  f1-score   support

           0       0.71      0.75      0.73       289
           1       0.72      0.67      0.70       274

    accuracy                           0.71       563
   macro avg       0.71      0.71      0.71       563
weighted avg       0.71      0.71      0.71       563



In [ ]:
predictions.predictions[0][0]